# Model I/O

任何语言模型应用程序的核心元素都是模型。 LangChain 为您提供了与任何语言模型交互的构建块。 
* 提示(Prompts)：模板化、动态选择和管理模型输入 
* 语言模型(Language models)：通过通用接口调用语言模型 
* 输出解析器(Output parsers)：从模型输出中提取信息


![](https://python.langchain.com/assets/images/model_io-1f23a36233d7731e93576d6885da2750.jpg)


## Prompts

编程模型的新方法是通过提示(Prompt)。提示(Prompt)是指模型的输入。此输入通常由多个组件构成。 LangChain 提供了几个类和函数来简化提示的构建和使用。 
* 提示(Prompt)模板：参数化模型输入 
* 示例选择器：动态选择要包含在提示中的示例


### Prompt Templates


In [1]:
from langchain import PromptTemplate

template = """/
You are a naming consultant for new companies.
What is a good name for a company that makes {product}?
"""

prompt = PromptTemplate.from_template(template)
prompt.format(product="colorful socks")

'/\nYou are a naming consultant for new companies.\nWhat is a good name for a company that makes colorful socks?\n'

In [2]:
from langchain import PromptTemplate

# An example prompt with no input variables
no_input_prompt = PromptTemplate(input_variables=[],
                                 template="Tell me a joke.")
no_input_prompt.format()
# -> "Tell me a joke."

# An example prompt with one input variable
one_input_prompt = PromptTemplate(input_variables=["adjective"],
                                  template="Tell me a {adjective} joke.")
one_input_prompt.format(adjective="funny")
# -> "Tell me a funny joke."

# An example prompt with multiple input variables
multiple_input_prompt = PromptTemplate(
    input_variables=["adjective", "content"],
    template="Tell me a {adjective} joke about {content}.")
multiple_input_prompt.format(adjective="funny", content="chickens")
# -> "Tell me a funny joke about chickens."

'Tell me a funny joke about chickens.'

> 如果您不想手动指定 input_variables，您也​​可以使用 from_template 类方法创建一个 PromptTemplate。 langchain 会根据传递的模板自动推断 input_variables。


#### Chat prompt template


In [5]:
from langchain.prompts import (
    ChatPromptTemplate,
    PromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import (AIMessage, HumanMessage, SystemMessage)

In [7]:
template = "You are a helpful assistant that translates {input_language} to {output_language}."
system_message_prompt = SystemMessagePromptTemplate.from_template(template)
human_template = "{text}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

chat_prompt = ChatPromptTemplate.from_messages(
    [system_message_prompt, human_message_prompt])

# get a chat completion from the formatted messages
chat_prompt.format_prompt(input_language="English",
                          output_language="French",
                          text="I love programming.").to_messages()

[SystemMessage(content='You are a helpful assistant that translates English to French.', additional_kwargs={}),
 HumanMessage(content='I love programming.', additional_kwargs={}, example=False)]

#### Connecting to a Feature Store

特征存储是传统机器学习的一个概念，它确保输入模型的数据是最新的和相关的。有关更多信息，请参见此处。 

在考虑将 LLM 应用程序投入生产时，这个概念非常重要。为了个性化 LLM 应用程序，您可能希望将 LLM 与有关特定用户的最新信息结合起来。特征存储是保持数据最新的好方法，LangChain 提供了一种将该数据与 LLM 结合的简单方法。 

在此笔记本中，我们将展示如何将提示模板连接到特征存储。基本思想是从提示模板内部调用特征存储来检索值，然后将这些值格式化为提示。


##### Feast

首先，我们将使用流行的开源特征存储框架 [Feast](https://github.com/feast-dev/feast)。 

这假设您已经运行了 README 中关于入门的步骤。我们将在入门部分构建该示例，并创建和 LLMChain 以向特定驱动程序写入有关其最新统计数据的注释。


In [ ]:
%pip install feast

In [2]:
from feast import FeatureStore
import pandas as pd
from datetime import datetime

entity_df = pd.DataFrame.from_dict({
    "driver_id": [1001, 1002, 1003, 1004],
    "event_timestamp": [
        datetime(2021, 4, 12, 10, 59, 42),
        datetime(2021, 4, 12, 8, 12, 10),
        datetime(2021, 4, 12, 16, 40, 26),
        datetime(2021, 4, 12, 15, 1, 12)
    ]
})

store = FeatureStore(repo_path="../my_feature_repo/feature_repo")

training_df = store.get_historical_features(
    entity_df=entity_df,
    features=[
        'driver_hourly_stats:conv_rate', 'driver_hourly_stats:acc_rate',
        'driver_hourly_stats:avg_daily_trips'
    ],
).to_df()

print(training_df.head())

# Train model
# model = ml.fit(training_df)

   driver_id           event_timestamp  conv_rate  acc_rate  avg_daily_trips
0       1001 2021-04-12 10:59:42+00:00   0.960894  0.269126              916
1       1002 2021-04-12 08:12:10+00:00   0.563219  0.632244              981
2       1003 2021-04-12 16:40:26+00:00   0.933522  0.890652               13
3       1004 2021-04-12 15:01:12+00:00   0.243195  0.410256              254
